In [15]:
import os, sys
import argparse

import xml.etree.ElementTree as ET

In [16]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci
import traci.constants as tc

net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [17]:
import curb

In [28]:
def main():
    """
    Main function that controls the simulation
    """

    road_network = curb.create_graph('seattle.net.xml')
    
    traci.parkingarea.subscribeContext('01-24-NE', tc.CMD_GET_VEHICLE_VARIABLE, 50, [tc.VAR_TYPE])
    traci.close()
    
    # create agent curbs
    curbs = []
    root = ET.parse(add_xml).getroot()
    for child in root.iter('additional'):
        for kid in child.iter('parkingArea'):
            curbs.append(kid.get('id'))
                     
    for curb_id in curbs:
        b = curb.smart_curb(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, 'passenger', road_network)
    
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()

        # if curb is still operating by time
        if traci.simulation.getTime():
            operating = True

        # look at each vehicle
        confirmed = False
        trigger = False

        # if the vehicle has not confirmed parking, plans to park at the smart curb
        if (not confirmed) and traci.vehicle.getNextStops('e_g60_31') and \
           "01-24-NE" in [item[2] for item in traci.vehicle.getNextStops('e_g60_1')]:

            # if vehicle type matches, the curb is open and the curb is not fully occupied
            if traci.vehicle.getVehicleClass('e_g60_31') == 'passenger' and \
               operating and \
               (int(traci.simulation.getParameter('01-24-NE', "parkingArea.occupancy")) < b.capacity):
                    confirmed = True
            else:
                # otherwise, the parking is rejected and need to trigger a reroute
                trigger = True

        if trigger:
            # dynamically reroute to the another lot
            traci.vehicle.rerouteParkingArea('e_g60_31', '01-24-SW')

        # in real decision, the curb should see a list of candidate vehicles
        # retrieve a list and then take that and other information (congestion on its edge) as state to make decision

        # print(traci.simulation.getParameter('ParkAreaA', "parkingArea.lane"))
        # print(traci.simulation.getParameter('ParkAreaA', "parkingArea.occupancy"))

    # enter False to disconnect before SUMO finishes
    traci.close()

    # flush out intermmediate results
    sys.stdout.flush()
    
    
    # backup
    ## junction listen
    ### traci.junction.subscribeContext('01-24', tc.CMD_GET_VEHICLE_VARIABLE, 100, 
                                ## [tc.VAR_TYPE, tc.VAR_ROUTE_ID, tc.VAR_LENGTH, 
                                ## tc.VAR_POSITION, tc.VAR_ROAD_ID, tc.VAR_LANEPOSITION])
            
    ## create a general curb
    ### a = curb.curb('seattle.add.xml', 'seattle.net.xml', '01-24-NE', 'passenger')
    
    # retrive curb listen
    ### if traci.junction.getContextSubscriptionResults('gneJ5'):
        ### print(traci.junction.getContextSubscriptionResults('gneJ5'))

In [29]:
traci.start(["sumo", "-c", "seattle.sumocfg"])
main()

 Retrying in 1 seconds


TraCIException: Command not implemented in sumo

In [27]:
traci.close()

# flush out intermmediate results
sys.stdout.flush()